## The `str` type.

In [ ]:
text = "quick brown fox jumps over the lazy dog."
print(type(text))

<class 'str'>


## Working with Unicode

In [ ]:
text = "El Niño".encode("utf-8")
print('original text: ', text)
print('text after decoding: ', text.decode("utf-8"))

original text:  b'El Ni\xc3\xb1o'
text after decoding:  El Niño


### Size of different strings

In [ ]:
import sys
string = 'hello'
print(sys.getsizeof(string))

# one byte encoding
print(sys.getsizeof(string+'!')-sys.getsizeof(string))

# 2-byte encoding
string2  = 'অ'
print(sys.getsizeof(string2+'অ')-sys.getsizeof(string2))
print(sys.getsizeof(string2))

# 4-byte encoding
string3 = '🐍'
print(sys.getsizeof(string3+'💻')-sys.getsizeof(string3))
print(sys.getsizeof(string3))

54
1
2
76
4
80


## Common Python string methods

### Character lengths and word lengths

In [ ]:
string = 'natural language processing for professionals'
print('character length of the sentence:', len(string))

words = string.split()
print('word length of the sentence:', len(words))

character length of the sentence: 45
word length of the sentence: 5


### Character frequency

In [ ]:
from collections import Counter

print(Counter('natural language processing for professionals'))

Counter({'a': 5, 's': 5, 'n': 4, 'r': 4, ' ': 4, 'o': 4, 'l': 3, 'g': 3, 'e': 3, 'u': 2, 'p': 2, 'i': 2, 'f': 2, 't': 1, 'c': 1})


### Pattern Search

In [ ]:
spam_string = "click on http://spam.com"

print('http' in spam_string)

True


In [ ]:
import re

def url_present(text):
    pattern = 'http.*\.com'
    if re.search(pattern, spam_string):
        return True
    else:
        return False

spam_string = "click on http://spam.com"
print(url_present(spam_string))

spam_string = "update on ticket"
print(url_present(spam_string))

True
False


### Strip whitespace

In [ ]:
string_with_whitespace = "    I am without whitespace. \n"
string_without_whitespace = "I am without whitespace."
print('Equality of the two strings: ',
  string_without_whitespace==string_with_whitespace)
print('Equality of the strings after performing strip on string_without_whitespace: ',
  string_with_whitespace.strip()==string_without_whitespace)

Equality of the two strings:  False
Equality of the strings after performing strip on string_without_whitespace:  True


### Splitting Strings

In [ ]:
document = "learning natural language processing"
print(document.split())

['learning', 'natural', 'language', 'processing']


### Joining list elements in a contiguous string

In [ ]:
tokens = ['learning', 'natural', 'language', 'processing']
print(" ".join(tokens))

learning natural language processing


### Case of the string

In [ ]:
string = "EDUCATIVE"
print('unique object identification of `string`', id(string))

lower_case = string.lower()
print('sentence in lower case: ', lower_case)
print('unique object identification of `lower_case`', id(lower_case))

upper_case = lower_case.upper()
print('sentence in upper case: ', upper_case)
print('unique object identification of `upper_case`', id(upper_case))

print('is a match with the original text: ', upper_case == string)

unique object identification of `string` 140295863579824
sentence in lower case:  educative
unique object identification of `lower_case` 140295854557232
sentence in upper case:  EDUCATIVE
unique object identification of `upper_case` 140295854558704
is a match with the original text:  True


## Pandas

In [ ]:
import pandas as pd
s1 = pd.Series(
    ['string processing using vanila python',
     'string processing in pandas']
)
print(s1)
print()
print('converting all the strings to uppercase')
print(s1.str.upper())
print()
print('converting all the strings to lowercase')
print(s1.str.lower())
print()
print('split all the sentences to words')
print(s1.str.strip().str.split())

0    string processing using vanila python
1              string processing in pandas
dtype: object

converting all the strings to uppercase
0    STRING PROCESSING USING VANILA PYTHON
1              STRING PROCESSING IN PANDAS
dtype: object

converting all the strings to lowercase
0    string processing using vanila python
1              string processing in pandas
dtype: object

split all the sentences to words
0    [string, processing, using, vanila, python]
1               [string, processing, in, pandas]
dtype: object


## Amazon reviews dataset

In [ ]:
import urllib.request as req
from urllib.parse import urlparse
import os
import progressbar
import zipfile
import gzip
import shutil
import json
import pandas as pd
import re
import string

pbar = None


def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

def wget(url):
    a = urlparse(url)
    filename = os.path.basename(a.path)
    if not os.path.isfile(filename):
        req.urlretrieve(url, filename, show_progress)
        print(f'downloaded to {filename}')
    else:
        print(f'file {filename} has already been downloaded')
    return filename

def unzip(filename, directory_to_extract_to=os.getcwd()):
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)
        print(f'extraction done {zip_ref.namelist()}')

def gunzip(gzfile, fout):
    with gzip.open(gzfile, 'rb') as f_in:
        with open(fout, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f'{gzfile} extracted to {fout}')


def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    df = {}
    i = 0
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


# map punctuation to space
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) 

def text_preprocessing(df):
    """
    Preprocess the text for better understanding
    
    """
    # trim the whitespace at the edges of the string
    df['reviewText'] = df['reviewText'].str.strip()

    # lowercase the text in the string
    df['reviewText'] = df['reviewText'].str.lower()

    # replace new line with a .

    df['reviewText'] = df['reviewText'].replace('\n', '.')

    # # remove the punctualtion in the string.
    # df['reviewText'] = df['reviewText'].str.replace('.', ' [EOS] ')
    # df['reviewText'] = df['reviewText'].apply(lambda text: text.translate(translator))
    # df['reviewText'] = df['reviewText'].str.replace('[EOS]', ' . ')

    return df


filename = wget("https://nlp.stanford.edu/data/glove.6B.zip")
unzip(filename)
Video_Games_5 = wget('http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz')
df = getDF(Video_Games_5)
df = df[['reviewText', 'overall']]
print(df.shape)

100% (862182613 of 862182613) |##########| Elapsed Time: 0:02:40 Time:  0:02:40


downloaded to glove.6B.zip
extraction done ['glove.6B.50d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt']


100% (154050105 of 154050105) |##########| Elapsed Time: 0:00:03 Time:  0:00:03


downloaded to Video_Games_5.json.gz
(497577, 2)


In [ ]:
import string

# map punctuation to space
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) 

def text_preprocessing(df):
    """
    Preprocess the text for better understanding
    
    """
    # trim the whitespace at the edges of the string
    df['reviewText'] = df['reviewText'].str.strip()

    # lowercase the text in the string
    df['reviewText'] = df['reviewText'].str.lower()

    # remove the punctualtion in the string.
    df['reviewText'] = df['reviewText'].apply(lambda text: text.translate(translator))

    return df

df = df.dropna()
df = df.drop_duplicates()
df = text_preprocessing(df)

In [ ]:
df.head()

,reviewText,overall
0,this game is a bit hard to get the hang of bu...,5.0
1,i played it a while but it was alright the st...,4.0
2,ok game,3.0
3,found the game a bit too complicated not what...,2.0
4,great game i love it and have played it since...,5.0
